# Problem Statement

Assume that you are working as a Data Analyst Intern with Uber.
Your first assignment as an intern here is to perform analysis and ML modelling on rides data recorded between 2009-01-01 and
2015-06-30.

In [1]:
import pandas as pd

# Load the dataset
file_path = '/content/uber_rides_data.xlsx - sample_train.csv'
df = pd.read_csv(file_path)

# Show the first few rows to get an idea of the data structure
df.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


# Dataset Discription

Great! The dataset has the following columns:

ride_id: Identifier for each ride

fare_amount: The fare amount for the ride

pickup_datetime: The date and time when the ride started

pickup_longitude: The longitude where the ride was picked up

pickup_latitude: The latitude where the ride was picked up

dropoff_longitude: The longitude where the ride was dropped off

dropoff_latitude: The latitude where the ride was dropped off

passenger_count: The number of passengers in the ride

# Data Cleaning

#  shape of given dataset


In [2]:
# Check the shape of the dataset
dataset_shape = df.shape
dataset_shape

(200000, 8)

 # Integer columns(by default) are given in the dataset

In [3]:
# Check the data types of the columns to identify integer columns
integer_columns = df.select_dtypes(include=['int']).columns
integer_columns_count = len(integer_columns)
integer_columns, integer_columns_count

(Index(['ride_id', 'passenger_count'], dtype='object'), 2)

# Missing values exists in 'dropoff_longitude' column

In [4]:
# Count the number of missing values in the 'dropoff_longitude' column
missing_values_dropoff_longitude = df['dropoff_longitude'].isnull().sum()
missing_values_dropoff_longitude

1

# Data type of ' pickup_datetime' feature in your data

In [5]:
# Check the data type of the 'pickup_datetime' column
pickup_datetime_dtype = df['pickup_datetime'].dtype
pickup_datetime_dtype

dtype('O')

#  Average fare amount

In [7]:
# Remove rows with null values
df_cleaned = df.dropna()

# Calculate the average fare amount
average_fare_amount = df_cleaned['fare_amount'].mean()
average_fare_amount

11.359891549457748

# Calculating distance between each pickup and dropoff points using Haversine formula and the median haversine distance between pickup and dropoff location

In [8]:
import numpy as np

# Function to calculate haversine distance between two points given their longitude and latitude
def haversine_distance(lon1, lat1, lon2, lat2):
    # Convert latitude and longitude from degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    # Earth's radius in kilometers (mean radius = 6371 km)
    R = 6371.0

    # Calculate the distance
    distance = R * c

    return distance

# Calculate the haversine distance for each ride
df_cleaned['haversine_distance'] = haversine_distance(df_cleaned['pickup_longitude'], df_cleaned['pickup_latitude'],
                                                      df_cleaned['dropoff_longitude'], df_cleaned['dropoff_latitude'])

# Calculate the median haversine distance
median_haversine_distance = df_cleaned['haversine_distance'].median()
median_haversine_distance


<ipython-input-8-41a9f0ef86d0>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['haversine_distance'] = haversine_distance(df_cleaned['pickup_longitude'], df_cleaned['pickup_latitude'],


2.1209923961833708


 #  Maximum haversine distance between pickup and dropoff location

In [10]:
# Calculate the maximum haversine distance in the cleaned dataset
max_haversine_distance = df_cleaned['haversine_distance'].max()
max_haversine_distance


16409.239135313168


# Rides have 0.0 haversine distance between pickup and dropoff location

In [11]:
# Count the number of rides with 0.0 haversine distance
zero_haversine_distance_count = df_cleaned[df_cleaned['haversine_distance'] == 0.0].shape[0]
zero_haversine_distance_count


5632

# Mean 'fare_amount' for rides with 0 haversine distance

In [12]:
# Calculate the mean 'fare_amount' for rides with 0 haversine distance
mean_fare_zero_distance = df_cleaned[df_cleaned['haversine_distance'] == 0.0]['fare_amount'].mean()
mean_fare_zero_distance


11.585317826704546

# Expert Opinion
The mean fare_amount for rides with a Haversine distance of 0.0 km is approximately
This is somewhat surprising because we would expect the fare amount for rides with zero distance to be much lower, possibly even zero if the ride was cancelled immediately after being booked.

This discrepancy could indicate some inconsistencies or errors in the dataset. Here are a few possible explanations:

**Data Entry Errors**: The coordinates for pickup and dropoff locations might have been incorrectly recorded as the same, leading to a zero Haversine distance but a non-zero fare amount.

**Cancelled Rides:** Some of these could be cancelled rides that still incurred a minimum charge. However, the mean fare amount seems too high for this to be the only explanation.

Wait Time: The fare could include waiting time, not just distance traveled. However, this is less likely to be the sole reason for the high mean fare amount for zero-distance rides.

#  Maximum 'fare_amount' for a ride

In [13]:
# Calculate the maximum 'fare_amount' in the cleaned dataset
max_fare_amount = df_cleaned['fare_amount'].max()
max_fare_amount


499.0

#  Haversine distance between pickup and dropoff location for the costliest ride

In [14]:
# Find the haversine distance for the ride with the maximum 'fare_amount'
haversine_distance_max_fare = df_cleaned[df_cleaned['fare_amount'] == max_fare_amount]['haversine_distance'].values[0]
haversine_distance_max_fare


0.0007899213191009993

# Expert Opinion
The Haversine distance between the pickup and dropoff location for the costliest ride is approximately
0.00079
0.00079 km, which is almost negligible.

This situation is indeed puzzling and could indicate some inconsistencies or errors in the dataset. Here are a few possible explanations:

**Data Entry Errors:** It's possible that the coordinates for the pickup and dropoff locations were incorrectly recorded, leading to an almost zero Haversine distance but an extremely high fare amount.

**Special Cases:** The ride could be a special case where the fare is not solely based on the distance traveled. However, the extremely high fare amount for almost zero distance is still hard to justify in most scenarios.

Fraudulent Activities:Such inconsistencies in the data could also be indicative of fraudulent activities, although this is purely speculative without further information.

# Rides were recorded in the year 2014

In [15]:
# Convert 'pickup_datetime' to datetime format for easier time-based filtering
df_cleaned['pickup_datetime'] = pd.to_datetime(df_cleaned['pickup_datetime'])

# Extract the year from the 'pickup_datetime' column
df_cleaned['pickup_year'] = df_cleaned['pickup_datetime'].dt.year

# Count the number of rides recorded in the year 2014
rides_in_2014 = df_cleaned[df_cleaned['pickup_year'] == 2014].shape[0]
rides_in_2014


<ipython-input-15-c2ca5b62c422>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['pickup_datetime'] = pd.to_datetime(df_cleaned['pickup_datetime'])
<ipython-input-15-c2ca5b62c422>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['pickup_year'] = df_cleaned['pickup_datetime'].dt.year


29968

In [16]:
# Extract the year from the 'pickup_datetime' column in the original dataset
# First, convert 'pickup_datetime' to datetime format for easier time-based filtering
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Extract the year
df['pickup_year'] = df['pickup_datetime'].dt.year

# Count the number of rides recorded in the year 2014 in the original dataset
rides_in_2014_original = df[df['pickup_year'] == 2014].shape[0]
rides_in_2014_original


29968

# Rides were recorded in the first quarter of 2014

In [17]:
# Filter the dataset for rides in the first quarter (Jan, Feb, Mar) of 2014
rides_in_2014_Q1 = df[(df['pickup_year'] == 2014) & (df['pickup_datetime'].dt.month <= 3)].shape[0]
rides_in_2014_Q1


7687

# Day of the week in September 2010, maximum rides were recorded

In [18]:
# Import the calendar module to map day numbers to day names
import calendar

# Filter the dataset for rides in September 2010
rides_in_sep_2010 = df[(df['pickup_year'] == 2010) & (df['pickup_datetime'].dt.month == 9)]

# Extract the day of the week (0 = Monday, 1 = Tuesday, ..., 6 = Sunday)
rides_in_sep_2010['pickup_day_of_week'] = rides_in_sep_2010['pickup_datetime'].dt.dayofweek

# Count the number of rides for each day of the week
day_of_week_counts = rides_in_sep_2010['pickup_day_of_week'].value_counts().sort_index()

# Find the day with the maximum number of rides
max_rides_day_num = day_of_week_counts.idxmax()
max_rides_day_name = calendar.day_name[max_rides_day_num]

max_rides_day_name, day_of_week_counts


<ipython-input-18-0bc2e80fff90>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rides_in_sep_2010['pickup_day_of_week'] = rides_in_sep_2010['pickup_datetime'].dt.dayofweek


('Thursday',
 0    265
 1    322
 2    391
 3    457
 4    354
 5    362
 6    331
 Name: pickup_day_of_week, dtype: int64)

# Feature Engineering

To predict the fare amount given the input features (passenger_count, distance, and ride_week_day), we'll proceed with the following steps:

**Feature Engineering:** Create a feature for the day of the week ('ride_week_day') and keep the 'passenger_count' and 'haversine_distance' features as they are.

**Data Preprocessing:** One-hot encode the 'ride_week_day' feature since it's categorical.

**Data Splitting:** Perform a 70-30 split of the data.

**Model Training:** Train different machine learning algorithms on the training data.

**Model Evaluation:** Evaluate the models using the adjusted R square  value to find the algorithm that gives the least adjusted R square value.

In [19]:
# Add a 'ride_week_day' feature to the cleaned dataset, representing the day of the week
# Note: The day of the week with Monday=0, Sunday=6
df_cleaned['ride_week_day'] = df_cleaned['pickup_datetime'].dt.dayofweek

# Drop rows with unrealistic values as identified in the data cleaning steps
# 1. Remove rows where the 'fare_amount' is negative
# 2. Remove rows where the 'passenger_count' is either 0 or unreasonably high (greater than 8)
# 3. Remove rows with unrealistic 'haversine_distance' values (greater than 200 km or zero distance)
df_cleaned_filtered = df_cleaned[(df_cleaned['fare_amount'] > 0) &
                                 (df_cleaned['passenger_count'] > 0) &
                                 (df_cleaned['passenger_count'] <= 8) &
                                 (df_cleaned['haversine_distance'] > 0) &
                                 (df_cleaned['haversine_distance'] <= 200)]

# Show the first few rows of the dataset after adding 'ride_week_day'
df_cleaned_filtered[['fare_amount', 'passenger_count', 'haversine_distance', 'ride_week_day']].head()


<ipython-input-19-acd53a0d3e1a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['ride_week_day'] = df_cleaned['pickup_datetime'].dt.dayofweek


,fare_amount,passenger_count,haversine_distance,ride_week_day
0,7.5,1,1.683323,3
1,7.7,1,2.457590,4
2,12.9,1,5.036377,0
3,5.3,3,1.661683,4
4,16.0,5,4.475450,3


In [20]:
# One-hot encode the 'ride_week_day' feature
df_encoded = pd.get_dummies(df_cleaned_filtered, columns=['ride_week_day'], prefix='day', drop_first=True)

# Show the first few rows of the dataset after one-hot encoding
df_encoded.head()


,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_distance,pickup_year,day_1,day_2,day_3,day_4,day_5,day_6
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323,2015,0,0,1,0,0,0
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590,2009,0,0,0,1,0,0
2,44984355,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377,2009,0,0,0,0,0,0
3,25894730,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683,2009,0,0,0,1,0,0
4,17610152,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450,2014,0,0,1,0,0,0


# Train-Test Split

In [21]:
from sklearn.model_selection import train_test_split

# Features and target variable
X = df_encoded[['passenger_count', 'haversine_distance', 'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6']]
y = df_encoded['fare_amount']

# Split the data into training and test sets (70-30 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Check the shape of the training and test sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((135255, 8), (57967, 8), (135255,), (57967,))

# Model Building

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor  # Use KNeighborsRegressor for regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Initialize models
linear_reg = LinearRegression()
decision_tree = DecisionTreeRegressor(random_state=42)
random_forest = RandomForestRegressor(random_state=42)
knn_regressor = KNeighborsRegressor(n_neighbors=5)  # Initialize KNeighborsRegressor

# List of models
models = [linear_reg, decision_tree, random_forest, knn_regressor]  # Add KNN to the list
model_names = ['Linear Regression', 'Decision Tree', 'Random Forest', 'KNN Regressor']  # Update model names

# Dictionary to store R2 and adjusted R2 values
r2_scores = {}

# Train and evaluate each model
for model, name in zip(models, model_names):
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    if isinstance(model, KNeighborsRegressor):  # Handle KNeighborsRegressor differently
        # Since KNN is a regression model, directly calculate R2 score
        r2 = r2_score(y_test, y_pred)
        adjusted_r2 = r2  # Adjusted R2 is not relevant for KNN
    else:
        # Evaluate other regression models using your existing code
        r2 = r2_score(y_test, y_pred)
        n = X_test.shape[0]
        p = X_test.shape[1]
        adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

    # Store the R2 and adjusted R2 values
    r2_scores[name] = {'R2': r2, 'Adjusted_R2': adjusted_r2}

r2_scores


{'Linear Regression': {'R2': 0.5884691438726917,
  'Adjusted_R2': 0.5884123398620458},
 'Decision Tree': {'R2': 0.5246576322977891,
  'Adjusted_R2': 0.5245920203211574},
 'Random Forest': {'R2': 0.6873701476475647,
  'Adjusted_R2': 0.6873269950401797},
 'KNN Regressor': {'R2': 0.7062747650233034,
  'Adjusted_R2': 0.7062747650233034}}

# Conclusion

In this code, we explored the performance of different regression models on a given dataset. We trained and evaluated four different models: Linear Regression, Decision Tree, Random Forest, and K-Nearest Neighbors (KNN) Regressor.

The R2 (coefficient of determination) and adjusted R2 scores were used as evaluation metrics to assess the goodness of fit for each model. The R2 score indicates how well the model captures the variance in the target variable, while the adjusted R2 takes into account the number of predictors and provides a more accurate measure of model performance for models with different numbers of features.

After training and evaluation, the following conclusions were drawn:

**Linear Regression:** The Linear Regression model showed a moderate R2 score, indicating that it explained a reasonable amount of variance in the target variable. The adjusted R2 was calculated to account for the number of features.

**Decision Tree:** The Decision Tree model demonstrated variable performance, as it can capture complex relationships within the data. The R2 and adjusted R2 scores highlighted the model's ability to fit the training data closely, but it might lead to overfitting on certain datasets.

**Random Forest:** The Random Forest model, an ensemble of decision trees, generally performed well and provided a higher R2 score compared to a single decision tree. The adjusted R2 accounted for the ensemble's complexity.

**KNN Regressor:** The KNN Regressor, which predicts target values based on neighboring data points, was included for evaluation. Unlike linear regression and decision trees, KNN's performance was measured solely by the R2 score, as adjusted R2 is not applicable due to its different nature. The KNN Regressor showed its strengths in capturing localized patterns in the data.